In [ ]:
!pip install transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer

     |████████████████████████████████| 7.6 MB 3.1 MB/s eta 0:00:01     |████▍                           | 1.0 MB 3.1 MB/s eta 0:00:03     |█████████▋                      | 2.3 MB 3.1 MB/s eta 0:00:02     |████████████████▍               | 3.9 MB 3.1 MB/s eta 0:00:02     |███████████████████████████████▉| 7.6 MB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 23.9 MB/s eta 0:00:01
     |████████████████████████████████| 7.8 MB 28.2 MB/s eta 0:00:01     |█▋                              | 399 kB 28.2 MB/s eta 0:00:01     |████████▌                       | 2.1 MB 28.2 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 268 kB 27.9 MB/s eta 0:00:01
     |████████████████████████████████| 774 kB 40.3 MB/s eta 0:00:01
     |████████████████████████████████| 173 kB 43.7 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

DATASET FORMAT FOR DATES

In [ ]:
from torch.utils.data import Dataset
import json

class Date_data(Dataset):
    def __init__(self, path:str, tokenizer):
        f = open(path , "r")
        self.data = str(f.read()).split("\n")
        self.data.pop(-1)
        self.X = []
        for ind,item in enumerate(self.data):
            item=item.replace('\'', '')
            self.X.append(item.split(","))

        #print(self.X)
        for idx, i in enumerate(self.X):
            try:
                self.X[idx] = "<startofstring> "+i[0]+" <bot>: "+i[1]+" <endofstring>"
            except:
                break

        # self.X = self.X[:400]

        self.X.pop(-1)
        print(self.X[0])
        print(self.X[-1])
        print(len(self.X))

        self.X_encoded = tokenizer(self.X,truncation=True,padding=True , return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>",
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])

1

# Model

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch

def train(chatData, model, optim):

    epochs = 5

    for i in tqdm.tqdm(range(epochs)):
        for X, a in chatData:
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            loss.backward()
            optim.step()
        torch.save(model.state_dict(), "model_state_date.pt")
        print(infer("may 20 2034"))

def infer(inp):
    inp = "<startofstring> "+inp+" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a,pad_token_id=tokenizer.eos_token_id ,max_length= 20 )
    output = tokenizer.decode(output[0])
    return output


device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"


model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model = model.to(device)


date_data_ub = Date_data("Data/Assignment2aDataset.txt", tokenizer)
date_data =  DataLoader(date_data_ub, batch_size=64)

model.train()

optim = Adam(model.parameters(), lr=1e-3)



You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50261. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


<startofstring> may 20 2034 <bot>:  2034-05-20 <endofstring>
<startofstring> june 11 2070 <bot>:  2070-06-11 <endofstring>
39999


In [ ]:
# print("TRAINING")
# train(date_data, model, optim)

In [ ]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pytorch_total_params

124442880

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device

'cuda'

LOAD THE SAVED MODEL

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
model.load_state_dict(torch.load("Data/model_state_date.pt"),strict=False)

<All keys matched successfully>

# INFERING OVER COMPLETE DATA

In [ ]:
n=input()
while n!="close":
  print(infer(n))
  n=input()

<startofstring> 22nd may 2022 <bot>: 2022-05-22 <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<startofstring> 23rd january 2200 <bot>: 2023-01-23 <endofstring> <pad> <pad> <pad> <pad> <pad>
<startofstring> 2022 23rd may <bot>: 2022-05-23 <endofstring> <pad> <pad> <pad> <pad> <pad> <pad>


In [ ]:
from tqdm import tqdm
def remove_extra(l):
  l=tokenizer(l)["input_ids"]
  # print(l)
  # print(tokenizer.decode(l))

  i=l.index(50260)
  l=l[i+1:]
  l=[item for item in l if item not in [50257,50258,50259,50260]]

  # print(l)

  return tokenizer.decode(l)


date_pred=[]
date_actual=[]
human_date=[]
tp=0

for item in tqdm(date_data_ub.data):
  item=item.replace('\'', '')
  temp=item.split(",")
  #print(temp)
  human_date.append(temp[0])
  date_actual.append(temp[1].strip())
  date_pred.append(remove_extra(infer(temp[0])))
  tp+=(date_actual[-1]==date_pred[-1])

print(f"Human understandable date                 : {human_date}")
print(f"Machine understandable corresponding date : {date_actual}")
print("")
print(f"Finetuned GPT-2 generated date            : {date_pred}")

100%|██████████| 3/3 [00:00<00:00,  3.24it/s]

Human understandable date                 : ['may 20 2034', '9 may 1630', '15/03/2014']
Machine understandable corresponding date : ['2034-05-20', '1630-05-09', '2014-03-15']

Finetuned GPT-2 generated date            : ['2034-05-20', '1630-05-09', '2014-03-15']


In [ ]:
f"Fraction of examples correctly predicted by GPT-2 =  {tp/len(date_data_ub)}"

'Fraction of examples correctly predicted by GPT-2 =  0.9673991849796245'